<a href="https://colab.research.google.com/github/otopiachka/markdown-doc/blob/master/fashion_mnist_keras_tuner_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Оптимизация гиперпараметров нейросети с помощью [Keras Tuner](https://github.com/keras-team/keras-tuner)

Учебный курс "[Программирование нейросетей на Python](https://www.asozykin.ru/courses/nnpython)".

Чтобы запускать и редактировать код, сохраните копию этого ноутбука себе (File->Save a copy in Drive...). Свою копию вы сможете изменять и запускать.

Не забудьте подключить GPU, чтобы сеть обучалась быстрее (Runtime -> Change Runtime Type -> Hardware Accelerator -> GPU).



## Гиперпараметры обучения нейронной сети

- Количество слоев нейронной сети
- Количество нейронов в каждом слое
- Функции активации, которые используются в слоях
- Тип оптимизатора при обучении нейронной сети
- Количество эпох обучения

## Установка Keras Tuner

In [34]:
pip install -U keras-tuner

Requirement already up-to-date: keras-tuner in /usr/local/lib/python3.6/dist-packages (1.0.2)


## Подключаем нужные пакеты

In [35]:
%tensorflow_version 2.x
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import utils
from google.colab import files
from kerastuner.tuners import RandomSearch, Hyperband, BayesianOptimization
import numpy as np

## Подготовка данных для обучения сети

In [36]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [37]:
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train / 255 
x_test = x_test / 255 
y_train = utils.to_categorical(y_train, 10)
y_test = utils.to_categorical(y_test, 10)

## Задаем функцию создания нейронной сети

In [62]:
def build_model(hp):
    model = Sequential()
    activation_choice = hp.Choice('activation', values=['relu', 'sigmoid', 'tanh', 'elu', 'selu'])    
    model.add(Dense(units=hp.Int('units_input',    # Полносвязный слой с разным количеством нейронов
                                   min_value=800,    # минимальное количество нейронов - 128
                                   max_value=19200,   # максимальное количество - 1024
                                   step=800),
                    input_dim=784,
                    activation=activation_choice))
    model.add(Dense(units=hp.Int('units_hidden',        
                                   min_value=128,   
                                   max_value=1200,   
                                   step=32),
                    activation=activation_choice))    
    model.add(Dense(10, activation='softmax'))
    model.compile(
        optimizer=hp.Choice('optimizer', values=['Adam','Adamax','rmsprop','SGD']),
        loss='categorical_crossentropy',
        metrics=['accuracy'])
    return model

## Создаем tuner

Доступные типы тюнеров: 
- RandomSearch - случайный поиск.
- Hyperband - алгоритм оптимизации на основе многорукого бандита, Li, Lisha, and Kevin Jamieson. ["Hyperband: A Novel Bandit-Based Approach to Hyperparameter Optimization."Journal of Machine Learning Research 18 (2018): 1-52](http://jmlr.org/papers/v18/16-558.html).
- BayesianOptimization - [байесовская оптимизация](https://en.wikipedia.org/wiki/Bayesian_optimization).

In [63]:
tuner = BayesianOptimization(
    build_model,                 # функция создания модели
    objective='val_accuracy',    # метрика, которую нужно оптимизировать - 
                                 # доля правильных ответов на проверочном наборе данных
    max_trials=80,               # максимальное количество запусков обучения 
    directory='test_directory'   # каталог, куда сохраняются обученные сети  
    )

## Запускаем подбор гиперпараметров

Пространство поиска

In [64]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'sigmoid', 'tanh', 'elu', 'selu'], 'ordered': False}
units_input (Int)
{'default': None, 'conditions': [], 'min_value': 800, 'max_value': 19200, 'step': 800, 'sampling': None}
units_hidden (Int)
{'default': None, 'conditions': [], 'min_value': 128, 'max_value': 1200, 'step': 32, 'sampling': None}
optimizer (Choice)
{'default': 'Adam', 'conditions': [], 'values': ['Adam', 'Adamax', 'rmsprop', 'SGD'], 'ordered': False}


Подбор гиперпараметров

In [65]:
tuner.search(x_train,                  # Данные для обучения
             y_train,                  # Правильные ответы
             batch_size=400,           # Размер мини-выборки
             epochs=80,                # Количество эпох обучения 
             validation_split=0.2,     # Часть данных, которая будет использоваться для проверки
             )

Trial 80 Complete [00h 02m 24s]
val_accuracy: 0.902916669845581

Best val_accuracy So Far: 0.9055833220481873
Total elapsed time: 03h 19m 02s
INFO:tensorflow:Oracle triggered exit


## Выбираем лучшую модель

In [66]:
tuner.results_summary(num_trials=1)

Results summary
Results in test_directory/untitled_project
Showing 1 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
activation: relu
units_input: 1600
units_hidden: 1088
optimizer: rmsprop
Score: 0.9055833220481873


Получаем три лучших модели

In [67]:
models = tuner.get_best_models(num_models=1)
models

Оцениваем качество модели на тестовых данных

In [68]:
for model in models:
  model.summary()
  model.evaluate(x_test, y_test)
  print() 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1600)              1256000   
_________________________________________________________________
dense_1 (Dense)              (None, 1088)              1741888   
_________________________________________________________________
dense_2 (Dense)              (None, 10)                10890     
Total params: 3,008,778
Trainable params: 3,008,778
Non-trainable params: 0
_________________________________________________________________
313/313 [==============================] - 1s 2ms/step - loss: 0.8276 - accuracy: 0.9008



In [69]:
scores = model.evaluate(x_test, y_test)
print("Доля верных ответов на тестовых данных, в процентах:", round(scores[1] * 100, 4))

313/313 [==============================] - 1s 2ms/step - loss: 0.8110 - accuracy: 0.9007
Доля верных ответов на тестовых данных, в процентах: 90.07
